In [2]:
# learn an estimate of the reward function, ie Rhat = f(s,a)

In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import cPickle as pickle

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
train_data = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [6]:
val_data = pd.read_csv('../data/rl_val_data_final_cont.csv')

In [7]:
test_data = pd.read_csv('../data/rl_test_data_final_cont.csv')

In [8]:
train_data.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,mortality_90d,Weight_kg,...,median_dose_vaso,max_dose_vaso,input_total_tev,input_4hourly_tev,output_total,output_4hourly,cumulated_balance_tev,vaso_input,iv_input,reward
0,0.000000,3,7245052800,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.797351,0.939195,0.589916,0.750908,0.554500,0.0,4.0,0.125000
1,0.222560,3,7245067200,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.831780,0.934543,0.674384,0.819589,0.580033,0.0,4.0,0.657321
2,0.356608,3,7245081600,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.833222,0.656575,0.765423,0.939329,0.555033,0.0,2.0,1.367788
3,0.452837,3,7245096000,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834033,0.603831,0.783597,0.847073,0.545700,0.0,2.0,1.199099
4,0.527957,3,7245110400,0.0,0.412568,0.0,0.0,0,1,0.262712,...,0.0,0.0,0.834836,0.603831,0.794059,0.811583,0.539533,0.0,2.0,1.057596


In [35]:
def preproc(df):
    df_in = df.copy()
    keep_arr = list(np.loadtxt('../data/state_features.txt', dtype=str))
    keep_arr.append('iv_input')
    keep_arr.append('vaso_input')
    df_in = df_in[np.abs(df_in['reward']) >=10]
    features = df_in[keep_arr].values
    labels = df_in['reward'].values/15.0
    return features,labels

In [36]:
def batch_sample(batch_size, features, labels):
    idx = np.random.choice(np.arange(len(features)), batch_size, replace=False)
    return (np.vstack(features[idx]), labels[idx])

In [37]:
train_feat, train_labels = preproc(train_data)
val_feat, val_labels = preproc(val_data)
test_feat, test_labels = preproc(test_data)

In [38]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(train_feat, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
# for each state: if not terminal, deterministically get reward. if terminal, use clf to predict outcome,
# and estimated reward = 15 * outcome


In [15]:
feat_names = list(np.loadtxt('../data/state_features.txt', dtype=str))

In [ ]:
envmodel_save_dir = './env_model_regression/'
est_state_val = pickle.load(open( envmodel_save_dir + "est_next_states_val.p", "rb" ))
est_state_test =  pickle.load(open( envmodel_save_dir + "est_next_states_test.p", "rb" ))

In [ ]:
# Get the relevant fields from the estimated next states
lact_index = feat_names.index('Arterial_lactate')
sofa_index = feat_names.index('SOFA')

val_data['est_lactate'] = est_state_val[lact_index]
val_data['est_sofa'] = est_state_val[sofa_index]

test_data['est_lactate'] = est_state_test[lact_index]
test_data['est_sofa'] = est_state_test[sofa_index]

In [ ]:
# Run the reward estimator
# Some constants taken from preproc/new_rewards:
c0 = -0.1/4
c1 = -0.5/4
c2 = -2
def reward_estimator(df):
    rewards = []
    for count,i in enumerate(df.index):
        if count == len(df) - 1 or df.loc[i, 'icustayid'] != df.loc[df.index[count+1], 'icustayid']:
            feat = df[feat_names]
            est_outcome = clf.predict(feat)
            if est_outcome == 0:
                rewards.append(-15)
            else:
                rewards.append(15)
        else:
            lact_now = df.loc[i, 'Arterial_lactate']
            sofa_now = df.loc[i, 'SOFA']
            lact_next = df.loc[i, 'est_lactate']
            sofa_next = df.loc[i, 'est_sofa']
            r = 0
            if sofa_next == sofa_now and sofa_next != 0:
                reward += c0
            reward += c1*(sofa_next-sofa_now)
            reward += c2*np.tanh(lact_next - lact_now)
            rewards.append(reward
    return np.array(rewards)

In [ ]:
val_rewards = reward_estimator(val_data)
test_rewards = reward_estimator(test_data)

In [55]:
with open(r"val_rewards.p", "wb") as f:
    pickle.dump(val_rewards, f)

In [56]:
with open(r"test_rewards.p", "wb") as f:
    pickle.dump(test_rewards, f)